In [1]:
import matplotlib        as mpl
import matplotlib.pyplot as plt
import numpy             as np
import pickle

In [2]:
def update_graph(child,new_parent,graph):
    #get the child's old parent
    old_parent = graph[child]
    
    #safety valve to limit the while loop
    graph_size = len(graph)
    counter    = 0
    
    #assuming that the new_parent is further up in the tree then
    #we simply loop until the child equals the new_parent
    #which is the state when a node self-refers, which is a property of it
    #being a top node in the tree
    while child != new_parent and counter < graph_size:
        graph[child] = new_parent
        child        = old_parent
        old_parent   = graph[child]
        counter += 1

def find_root(k,proper_labels):
    #final clean up for those proper labels that weren't 
    #collapsed when there is a linker
    counter = 0
    while(proper_labels[k] != k and counter < len(proper_labels)):
        k = proper_labels[k]
        counter = counter + 1
    return k

def init_site_labels(rows,cols):
    proper_labels  = {-1:0,0:0}
    cluster_labels = {}
    for i in range(rows):
        for j in range(cols):
            cluster_labels[i*cols + j] = -1
    return cluster_labels, proper_labels  


def hoshen_kopleman(lattice):
    largest_label = 1
    rows, cols    = lattice.shape

    cluster_labels, proper_labels = init_site_labels(rows,cols)

    #forward sweep
    for i in range(rows):
        for j in range(cols):
            curr              = lattice[i,j]
            curr_linear_index = i*cols + j
            if curr != 0:  #current cell occupied - only do the work if so
                up_index          = max(0,i-1)
                left_index        = max(0,j-1)
                up_linear_index   = up_index*cols + j     if i != 0 else 0
                left_linear_index = i*cols + left_index   if j != 0 else 0
                up                = lattice[up_index,j]   if i != 0 else 0
                left              = lattice[i,left_index] if j != 0 else 0
                if up == 0 and left == 0: #site is an island (so far)
                    proper_labels[largest_label]      = largest_label
                    cluster_labels[curr_linear_index] = largest_label
                    largest_label       += 1
                elif up != 0 and left == 0: #site linked to the one above
                    cluster_labels[curr_linear_index] = cluster_labels[up_linear_index]
                elif up == 0 and left != 0: #site linked to the one to the left
                    cluster_labels[curr_linear_index] = cluster_labels[left_linear_index]
                elif up != 0 and left != 0: #site is a linker
                    up_proper_label         = proper_labels[cluster_labels[up_linear_index]]
                    left_proper_label       = proper_labels[cluster_labels[left_linear_index]]
                    root_cluster_label      = min(up_proper_label,left_proper_label)
                    proximate_cluster_label = max(up_proper_label,left_proper_label)
                    cluster_labels[curr_linear_index] = proximate_cluster_label
                    update_graph(proximate_cluster_label,root_cluster_label,proper_labels)


    #collapse the proper labels so that there is only one hop to the top of the tree
    collapsed_proper_labels = {}
    for k in proper_labels:
        collapsed_proper_labels[k] = find_root(k,proper_labels)

    return cluster_labels, collapsed_proper_labels

In [14]:
def analyze_lattice(mat):
    #determine the size of the lattice
    L, M = mat.shape
    
    #run HK on the lattice
    cluster_labels, proper_labels = hoshen_kopleman(mat)
    
    #find the unique cluster labels and the number of unique clusters
    set_unique_cluster_labels = set(proper_labels.values())
    num_unique_clusters       = len(set_unique_cluster_labels)
    
    #make a hash/association called 'translate' that simply renumbers
    #the unique cluster labels in sequential order.  This is needed
    #when a union forces a later cluster label to be skipped when it is 
    #discovered that that cluster is actually part of a previous one
    translate = {}
    for i,k in enumerate(set_unique_cluster_labels):
        translate[k] = i
    
    #allocate an array for cluster labels - same size of the lattice but is
    #flattened to be a 1-D array
    Cs = np.zeros(L*M,dtype=int)

    #load the various Cs components with the proper cluster label
    #and then reform as a lattice
    for i,k in enumerate(cluster_labels): 
        Cs[i] = translate[proper_labels[cluster_labels[k]]]
    Cs = Cs.reshape(L,M)    
    
    #check each cluster (by scanning of the 'c' clusters found) for a 
    #spanner defined as a cluster that either goes in a path between 
    #left and right edges or top and bottom ones
    n_s         = {}
    s_s         = {}
    P_inf       = 0 
    num_spanner = 0
    num_occ     = 0    
    for c in range(1,np.max(Cs)+1):
        #initially assume no spanning cluster
        span_flag        = 0
        #find the location of the current cluster with label 'c'
        cluster_spots    = np.where(Cs==c)
        #find the size of the current cluster with label 'c'
        c_size           = len(cluster_spots[0])
        #add to the total number of occupied sites the number of sites in
        #the current cluster with label 'c'
        num_occ          += num_occ + c_size
        #find the rows and columns covered by the current cluster with label 'c'
        rows             = cluster_spots[0]
        cols             = cluster_spots[1]
        #find the length and height of the current cluster with label 'c'
        row_span         = max(rows) - min(rows)
        col_span         = max(cols) - min(cols)
        #calculate the center of mass (rowsb,colsb) and the radius of gyration (s)
        rowsb            = rows - np.mean(rows)
        colsb            = cols - np.mean(cols)
        s                = np.sqrt(np.sum(rowsb*rowsb+colsb*colsb)/len(rows))        
        #check to see if it is a spanning cluster and set 'P_inf' appropriately
        if row_span == L - 1 and col_span == M - 1:
            #row & col  spanner
            num_spanner += 1
            P_inf       += c_size
            span_flag    = 1
        elif row_span == L - 1:
            #row spanner
            num_spanner += 1
            P_inf       += c_size
            span_flag    = 1
        elif col_span == M - 1:
            #col spanner
            num_spanner += 1
            P_inf       += c_size
            span_flag    = 1
        #create a label to distinguish spanning clusters from ordinary ones
        decorated_c_size = str(c_size)+'s'+str(span_flag)
        #accumulate 'n_s' which is not really the theoretical 'n(s)'
        if decorated_c_size in n_s.keys():
            n_s[decorated_c_size] += 1
        else:
            n_s[decorated_c_size] = 1
        #accumulate 's(s)'
        if decorated_c_size in s_s.keys():
            s_s[decorated_c_size] += s
        else:
            s_s[decorated_c_size] = s
   
    return num_spanner, P_inf/num_occ, n_s, Cs, s_s

In [4]:
def combine_n_s(trials):
    #find out the size of the lattice
    L, M = trials[0][3].shape
    
    n_s = {}
    num_trials = len(trials)
    for n in range(num_trials):
        curr_n_s = trials[n][2]  #third return val of analyze matrix is the n_s
        for k in curr_n_s.keys():
            if k in n_s.keys():
                n_s[k] += curr_n_s[k]/L/M/num_trials
            else:
                n_s[k] = curr_n_s[k]/L/M/num_trials
                
    return n_s #average numbers of clusters of size s

def unpack_n_s_data_to_arrays(n_s):
    sizes = []
    nums  = []
    for k in n_s.keys():
        temp = k.split('s')
        if int(temp[1]) == 0:
            sizes.append(int(temp[0]))
            nums.append(n_s[k])
        
    return np.array(sizes), np.array(nums)

def collect_data_over_N_trials(p_init,num_trials,L):
    Qs    = {}
    Pinfs = []
    for i in range(num_trials):
        mat        = np.zeros((L,L))
        probs      = np.random.rand(L,L)
        trues      = np.where(probs <= p_init)
        mat[trues] = 1
        temp       = analyze_lattice(mat)
        Qs[i] = (temp[0],temp[1],temp[2],temp[3],temp[4])
        Pinfs.append(temp[1])
        
    n_s = combine_n_s(Qs)
    
    return unpack_n_s_data_to_arrays(n_s), np.array(Pinfs)/num_trials, Qs

1. $ P_{\infty} \sim (p-p_c)^\beta $
1. $ S(p) \sim |p-p_c|^{-\gamma} $
1. $ \xi(p) \sim |p-p_c|^{-\nu} $
1. $ n_s \sim s^{-\tau} $

In [77]:
lattice_size = 8
num_samples  = 50
A_vlo,B_vlo,C_vlo = collect_data_over_N_trials(0.4   ,num_samples,lattice_size)
A_lo, B_lo, C_lo  = collect_data_over_N_trials(0.5   ,num_samples,lattice_size)
A_c,  B_c,  C_c   = collect_data_over_N_trials(0.5927,num_samples,lattice_size)
A_hi, B_hi, C_hi  = collect_data_over_N_trials(0.7   ,num_samples,lattice_size)
A_vhi,B_vhi,C_vhi = collect_data_over_N_trials(0.8   ,num_samples,lattice_size)

In [79]:
pwd

'C:\\Users\\byecs\\OneDrive\\Documents\\BlogWyrm'

In [78]:
data     = {'vlo':{'A':A_vlo,'B':B_vlo,'C':C_vlo},
            'lo' :{'A':A_lo, 'B':B_lo, 'C':C_lo},
            'c'  :{'A':A_c,  'B':B_c,  'C':C_c},
            'hi' :{'A':A_hi, 'B':B_hi, 'C':C_hi},
            'vhi':{'A':A_vhi,'B':B_vhi,'C':C_vhi}}
with open('data_8.pkl', 'wb') as file: 
    pickle.dump(data, file) 